In [ ]:
from TRSF import trsf
import numpy as np 
import matplotlib.pyplot as plt
import os
PATH_Cosmo_r0p0 = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_Continuum_Early_Science_COSMOS_r0p0.app.restored.circ.fits'
PATH_Cosmo_r1p2 = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_Continuum_Early_Science_COSMOS_r-1p2.app.restored.circ.fits'
PATH_XMMLSS_r0p0 = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_Continuum_Early_Science_XMMLSS_r0p0_circ.fits'
PATH_XMMLSS_r1p2 = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_Continuum_Early_Science_XMMLSS_r-1p2_circ.hires.fits'

# function to test if the file exists
def file_exists(path):
    if os.path.isfile(path):
        print('File exists : '+path)
        return True
    else:
        print('File does not exist : '+path)
        return False
    
file_exists(PATH_Cosmo_r0p0)
file_exists(PATH_Cosmo_r1p2)
file_exists(PATH_XMMLSS_r0p0)
file_exists(PATH_XMMLSS_r1p2)


In [ ]:
Cosmo_r0p0_data = trsf(PATH_Cosmo_r0p0,gaussian_fitting=False,region_expansion=False,cutup_img_size=500,sigma=5,smoothing=False,smooth_param=5)

In [ ]:
print(len(Cosmo_r0p0_data.catalogue))

In [ ]:
Cosmo_r1p2_data = trsf(PATH_Cosmo_r1p2,gaussian_fitting=False,region_expansion=False,cutup_img_size=500,sigma=5,smoothing=False,smooth_param=5)

In [ ]:
print(len(Cosmo_r1p2_data.catalogue))

In [ ]:
XMM_LSS_r1p2_data = trsf(PATH_XMMLSS_r1p2,gaussian_fitting=False,region_expansion=False,cutup_img_size=500,sigma=5,smoothing=False,smooth_param=5)

In [ ]:
print(len(XMM_LSS_r1p2_data.catalogue))

In [ ]:
XMM_LSS_r0p0_data = trsf(PATH_XMMLSS_r0p0,gaussian_fitting=False,region_expansion=False,cutup_img_size=500,sigma=5,smoothing=False,smooth_param=5)

In [ ]:
import skimage.measure as measure

def get_enclosing_mask(x, y, mask):
    '''
    Returns the mask of the enclosed area of the point (x,y) in the mask.
    '''
    
    # Ensure the point is inside the mask
    if not mask[y, x]:
        return None
    # Create a copy of the mask
    enclosed_mask = np.copy(mask)
    # Perform a flood fill starting from the point
    h, w = mask.shape
    stack = [(x, y)]
    while stack:
        x, y = stack.pop()
        if enclosed_mask[y, x]:
            enclosed_mask[y, x] = False
            if x > 0:
                stack.append((x - 1, y))
            if x < w - 1:
                stack.append((x + 1, y))
            if y > 0:
                stack.append((x, y - 1))
            if y < h - 1:
                stack.append((x, y + 1))
    
    return mask & (~enclosed_mask)

def plottings(cat,ptype,color,img):
    
    label = False
    for i in range(0,len(cat)):
        row = cat.iloc[i]
        if ptype=='bbox':
            bbox = row.bbox
            if bbox != None:
                xmin = bbox[0] - 1
                xmax = bbox[2] + 1
                ymin = bbox[1] - 1
                ymax = bbox[3] + 1
                if label == False:
                    plt.plot([ymin,ymin,ymax,ymax,ymin],[xmin,xmax,xmax,xmin,xmin],color=color,label='class 5')
                    label = True
                plt.plot([ymin,ymin,ymax,ymax,ymin],[xmin,xmax,xmax,xmin,xmin],color=color)
        if ptype == 'point':
            plt.plot(row.y,row.x,'+',color=color)
        
        if ptype == 'polygonwith':
            polygon = row.polygon
            try:
                if label == False:
                    plt.plot(polygon[:,1],polygon[:,0],color=color,label='class 2')
                    label = True
                else:
                    plt.plot(polygon[:,1],polygon[:,0],color=color)  
            except:
                #print('polygonwith Failed')
                continue
                #plottings(cat,'polygonwithout',color,img) 
                
        if ptype == 'polygonwithout':
        # we much create the polygons from the birth and death points
            try:
                birth = row.Birth
                death = row.Death
                mask = np.zeros(img.shape)
                mask = np.logical_or(mask, np.logical_and(img <= birth, img > death))
                #print(row.x_c,row.y_c)
                #plt.imshow(mask)
                ##plt.scatter(row.x_c,row.y_c,color='red')
                #plt.xlim(row.x_c-100,row.x_c+100)
                #plt.show()
                #plt.ylim(row.y_c-100,row.y_c+100)
                mask = get_enclosing_mask(int(row.x_c),int(row.y_c),mask)
                mask = mask.astype(int)

                contour = measure.find_contours(mask)[0]

                if label == False:
                    plt.plot(contour[:,1],contour[:,0],color=color,label='class 2')
                    label= True
                else:
                    plt.plot(contour[:,1],contour[:,0],color=color)   
            except:
                print(row)

In [ ]:
catalogue = XMM_LSS_r0p0_data.catalogue
image = XMM_LSS_r0p0_data.full_img
# change any 2 to 7
#catalogue['Class'][catalogue['Class'] == 2] = 7
catalogue['Class'][np.isnan(catalogue['Class'])] = 2
cat0 = catalogue[catalogue['Class'] == 0]
cat1 = catalogue[catalogue['Class'] == 1]
cat2 = catalogue[catalogue['Class'] == 2]
cat3 = catalogue[catalogue['Class'] == 3]
cat4 = catalogue[catalogue['Class'] == 4]
cat5 = catalogue[catalogue['Class'] == 5]

In [ ]:
cat2

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(image,origin='lower',cmap='gray',vmax=0.01)
#plottings(cat0,'polygonwithout',color='b',img=image)
#plottings(cat2,'polygonwith',color='r',img=image)
plottings(cat5,'polygonwithout',color='g',img=image)
#plottings(cat4,'polygonwithout',color='orange',img=image)
#plottings(cat3,'polygonwithout',color='purple',img=image)
#plottings(cat1,'polygonwithout',color='y',img=image)
plt.xlim(3000,4000)
plt.ylim(3000,4000)
plt.show()

In [ ]:
print(len(XMM_LSS_r0p0_data.catalogue))

In [ ]:
flux_tot = XMM_LSS_r0p0_data.catalogue.flux_tot.to_list()
flux_peak = XMM_LSS_r0p0_data.catalogue.flux_peak.to_list()

print(flux_tot)

In [ ]:
# open appropriate pybdsf catalog
PATH_XMM_LSS_r0p0_cat = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_CONTINUUM_Early_Science_XMMLSS_r0p0_circ.srl.fits'
PATH_XMM_LSS_r1p2_cat = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_CONTINUUM_Early_Science_XMMLSS_r-1p2_circ.hires.gaul.fits'
PATH_Cosmo_r0p0_cat = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_CONTINUUM_Early_Science_COSMOS_Level1.fits'
PATH_Cosmo_r1p2_cat = '/Users/rs17612/Documents/Radio_Data/MIGHTEE/MIGHTEE_CONTINUUM_Early_Science_COSMOS_r-1p2.app.restored.circ.gaul.fits'
# check path is correct
file_exists(PATH_XMM_LSS_r0p0_cat)
file_exists(PATH_XMM_LSS_r1p2_cat)
file_exists(PATH_Cosmo_r0p0_cat)
file_exists(PATH_Cosmo_r1p2_cat)

In [ ]:
from astropy.io import fits
import pandas as pd

def flux_hist(cat_path,trsf_flux):
    # open cat
    bdsf_df = pd.DataFrame(fits.open(cat_path)[1].data)
    print('Catalogue opened, Lenght: ', len(bdsf_df))
    bdsf_tot_flux = bdsf_df['Total_flux'].to_list()
    bdsf_peak_flux = bdsf_df['Peak_flux'].to_list()

    flux = np.log10(trsf_flux)
    bdsf_flux = np.log10(bdsf_tot_flux)

    plt.hist(flux, bins=20,fill=False, histtype='step', color='black', linewidth=2,label='TRSF')
    plt.hist(bdsf_flux, bins=20,fill=False, histtype='step', color='red', linewidth=2,label='pyBDSF')
    plt.title('Flux distribution')
    plt.xlabel('Flux_peak [Jy]')
    plt.ylabel('Number of components')
    plt.show()

In [ ]:
# opend fits catalogue
from astropy.io import fits
import pandas as pd
# open fits file
hdulist = fits.open(PATH_XMM_LSS_r0p0_cat)
bdsf_df = pd.DataFrame(hdulist[1].data)
print(len(bdsf_df))



In [ ]:
print(bdsf_df.columns)
flux_bdsf = bdsf_df['Peak_flux'].values


In [ ]:
# plot histogram of flux_tot
flux = np.log10(flux_peak)
bdsf_flux = np.log10(flux_bdsf)
plt.hist(flux, bins=20,fill=False, histtype='step', color='black', linewidth=2,label='TRSF')
plt.hist(bdsf_flux, bins=20,fill=False, histtype='step', color='red', linewidth=2,label='pyBDSF')
plt.title('XMMLSS r0p0 srl')
plt.xlabel('Flux_peak [Jy]')
plt.ylabel('Number of components')
plt.show()